In [1]:
#!pip install azureml-sdk

In [58]:
import azure.core
from azureml.core import Workspace, Environment,Experiment,ScriptRun
from azureml.core.compute import AmlCompute, ComputeInstance # for creating a compute cluster from python code
from azureml.core.compute import ComputeTarget
from azureml.core import ScriptRunConfig
from azureml.core.webservice import AciWebservice

## Creating Workspace

In [6]:
ws = Workspace.create("demows", resource_group="analytics",
                      subscription_id="a731217c-399f-4a77-9db6-81293ed043d6",
                      create_resource_group=True,
                      location='eastus2')

Deploying StorageAccount with name demowsstoragece5150448bf.
Deploying KeyVault with name demowskeyvault44de8d4abc.
Deployed KeyVault with name demowskeyvault44de8d4abc. Took 28.55 seconds.
Deploying AppInsights with name demowsinsights69ccc697e7.
Deployed AppInsights with name demowsinsights69ccc697e7. Took 74.7 seconds.
Deploying Workspace with name demows.
Deployed Workspace with name demows. Took 43.34 seconds.


In [7]:
#if we want to place this workspace into a local folder
ws.write_config('.azureml')

In [8]:
# see workspace details
ws.from_config()

Workspace.create(name='demows', subscription_id='a731217c-399f-4a77-9db6-81293ed043d6', resource_group='analytics')

In [9]:
# just an try and except block for workspace
#i.e if workspace is alrady present if not create

try:
    ws.write_config('.azureml')
except:
    ws = Workspace.create("demows", resource_group="analytics",
                      subscription_id="a731217c-399f-4a77-9db6-81293ed043d6",
                      create_resource_group=True,
                      location='eastus2')

## Create Compute Cluster

In [16]:
# Creating compute cluster will by default create 2 datastorages
try:
    cluster = ComputeTarget(ws, 'demo-cluster') # if cluster is already present
except:
    # creating a config file for the compute cluster
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2',
                                                  min_nodes=0,
                                                  max_nodes=6,
                                                  idle_seconds_before_scaledown=2400)
    # creating a compute cluster in workspace('ws') with above config
    cluster = ComputeTarget.create(ws,'demo-cluster',config)

## How can we submit code from local machine to azure

In [17]:
# shows how many compute targets are present
ws.compute_targets

{'demo-cluster': AmlCompute(workspace=Workspace.create(name='demows', subscription_id='a731217c-399f-4a77-9db6-81293ed043d6', resource_group='analytics'), name=demo-cluster, id=/subscriptions/a731217c-399f-4a77-9db6-81293ed043d6/resourceGroups/analytics/providers/Microsoft.MachineLearningServices/workspaces/demows/computes/demo-cluster, type=AmlCompute, provisioning_state=Succeeded, location=eastus2, tags={})}

In [19]:
# will show all datastorage
ws.datastores

{'workspaceworkingdirectory': {
   "name": "workspaceworkingdirectory",
   "container_name": "code-391ff5ac-6576-460f-ba4d-7e03433c68b6",
   "account_name": "demowsstoragece5150448bf",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspaceartifactstore': {
   "name": "workspaceartifactstore",
   "container_name": "azureml",
   "account_name": "demowsstoragece5150448bf",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspaceblobstore': {
   "name": "workspaceblobstore",
   "container_name": "azureml-blobstore-b0d997a6-518f-4486-91e5-d63d731cf8ff",
   "account_name": "demowsstoragece5150448bf",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspacefilestore': {
   "name": "workspacefilestore",
   "container_name": "azureml-filestore-b0d997a6-518f-4486-91e5-d63d731cf8ff",
   "account_name": "demowsstoragece5150448bf",
   "protocol": "https",
   "endpoint": "core.windows.net"
 }}

### Creating an Experiment/Job

In [20]:
exp = Experiment(ws,"demoexp")

In [22]:
# shows this recently created experiment is linked to perticular workspace
# note: here experiment is mean by job
exp

Name,Workspace,Report Page,Docs Page
demoexp,demows,Link to Azure Machine Learning studio,Link to Documentation


#### Creating a environment

In [50]:
# Just like in our local machine we create an environment we also need to create same in azure, to run the job.

env = Environment.from_conda_specification(name="my-env", file_path= "./envfile.yaml")


In [51]:
# without passing runtime arguments

pythonconfig = ScriptRunConfig(source_directory="./",script="Train.py",compute_target= "demo-cluster",environment=env)
execution = exp.submit(pythonconfig)

In [ ]:
# Passing Arguments at runtime

pythonconfig = ScriptRunConfig(source_directory="./",script="Train.py",compute_target= "demo-cluster",environment=env,
                              arguments=["--min_samples_leaf", 5,
                                         "--min_samples_split",7])
execution = exp.submit(pythonconfig)

#### Note: if we want logs in azure GUI we need to call some azure SDK in code. i.e azure.core import Run

### Register the Model

In [ ]:
# This will register the model in azureml in specified workspace
from azureml.core import Model
Model.register(ws,model_path=./diabeticmodel.pkl, model_name='diabeticmodel1')

## Deploy Model
1. Prepare the prediction script
2. Cerate Inference Environment: It may possible that while predicting we don't need some of the libraries which are needed     while training mode, so new environment will ne great for predtiction!
3. Define Inference Config
4. Choose the compute Target where you want to deploy your model
5. Deploy the model

### Note: we need to write source code for predicting which will include azure registered model location and model name

In [52]:
from azureml.core.model import InferenceConfig
from azureml.core.conda_dependencies import CondaDependencies

In [54]:
# Creating inference conda environment

predenv = CondaDependencies()

In [55]:
# specifing required libraries in conda enironment
predenv.add_conda_package('scikit-learn')
predenv.add_conda_package('azure-sdk')

In [56]:
# object of conda environment
predenv

In [57]:
# Creating yml file
env_file = './predenv.yml'
with open (env_file,"w") as f:
    f.write(predenv.serialize_to_string())

print("Saved dependency info in", env_file)

Saved dependency info in ./predenv.yml


In [ ]:
# Creating an infrence config
from azureml.core.model import InferenceConfig
inferconfig = InferenceConfig(runtime='python',source_directory='./', entry_script='score.py', conda_file=env_file)

In [ ]:
# We are creating compute target. Here we are creating azure compute instance.
# We can also do do it in aks

from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cup_cores=1, memory_gb=1)


In [ ]:
model = Model(ws, "diabeticmodel_instance")
deploy_infer = Model.delpoy(ws, "scorepredictiondemo",
                           models=[model],
                           inference_config = inference_config,
                           deployment_config = aciconfig,
                           deployment_target = predenv)